## Q1)

가설검정과 모델 분류와 연결지어 생각해보겠다. 
- 가설검정에서의 $H_0$는 모델 분류에서의 실제 정답, 위 예시에서는 실제 해당 고객이 가입 요건에 충족하는지 여부에 대한 것이다.
또한, 가설검정에서의 $H_0$의 기각 여부는 모델 분류에서의 분류결과이며, 위 예시에서는 해당 고객의 가입에 대한 승인 여부와 비슷하다. $H_0$를 기각하는 것은 Negative라고 판단하는 것과 일맥상통한다.  <br>
- 위 예시의 가입 승인 비율이 높은 것으로 보아, 가입 요건에 충족하지 않음에도 승인해준 경우에 해당되는 FP의 비율은 높지만 반대로 FN의 비율은 낮음을 알 수 있다. 이를 가설검정으로 적용해보면, **FP와 비슷한 맥락의 제 1종 오류는 높지만 FN과 관계가 깊은 제 2종 오류는 낮다**고 볼 수 있다.  

## Q2-1)
- Accurary(정확도) $= \frac{TP+TN}{TP+FN+FP+TN}$ 
 - 정확도는 모든 경우의 수 중에서 옳은 결정을 하는 것의 비율이며, 이때의 옳은 결정은 실제 Positive를 Positive로 분류하거나, Negative를 Negative로 분류하는 것이다. <br>
<br>
- Precision(정밀도) $= \frac{TP}{TP+FP}$
 - 정밀도는 모델이 Positive로 분류한 것 중에서 실제로도 Positive한 것의 비율이다. <br>
<br>
- Recall(재현율) $= \frac{TP}{TP+FN}$
 - 재현율은 실제 Positive한 것 중에서 모델이 Positive로 분류한 것의 비율이다. 

## Q2-2)
※ *성공확률의 threshold를 늘리면 그만큼 조건이 까다로워져서 FP가 낮아지고 FN이 높아진다.* <br> 
   *그대신 판단 기준이 까다로워진 만큼 Positive로 분류할 확률은 줄어든다.* 

- 중고등학교에서 아프다는 핑계로 야간자율학습을 빼고 조퇴하는 경우에 담당 교사의 판단을 예시로 들 수 있다. 학생들이 실제로 아픈 것과 아픈 척하는 것에 대해 담당 교사가 조퇴 여부에 옳은 판단을 내릴지와 연결지을 수 있다. FP를 예로 들면, 실제로는 학생이 야자를 째고 놀러가려고 아픈 척 연기를 하는 것인데 담당 교사가 진짜 아픈 줄 알고 조퇴를 시켜주는 경우가 해당된다. <br>
- 이때 성공확률의 treshold를 늘리는 것은, 교사의 판단 조건이 까다로워지는 것과 일맥상통하며 합리적이라고 할 수 있다. 꾀를 부려 야자를 상습적으로 째려고 하는 학생들을 바로잡아 올바른 방향으로 지도할 수 있기 때문이다. 

## Q3)

In [11]:
# 필요한 패키기 불러오기 
import torch 
import torch.nn as nn 
import torchvision.datasets as dsets 
import torchvision.transforms as transforms 
from torch.autograd import Variable 

In [12]:
# Hyper Parameters  
input_size = 784
num_classes = 10
num_epochs = 5
batch_size = 100
learning_rate = 0.001

In [18]:
# MNIST Dataset (Images and Labels) 

# 과적합 방지를 위해 train과 test set으로 나누기 
train_dataset = dsets.MNIST(root ='./mnist_data/',  
                            train = True,  
                            transform = transforms.ToTensor(),  # transforms.ToTensor():Image를 PyTorch로 변환
                            download = True) 
  
test_dataset = dsets.MNIST(root ='./mnist_data/',  
                           train = False,  
                           transform = transforms.ToTensor()) 
  
    
# Dataset Loader (Input Pipline) 
train_loader = torch.utils.data.DataLoader(dataset = train_dataset,  
                                           batch_size = batch_size,  
                                           shuffle = True)  # 전체 데이터에서 batch_size만큼 불러올때 무작위로 불러올지 여부 
  
test_loader = torch.utils.data.DataLoader(dataset = test_dataset,  
                                          batch_size = batch_size,  
                                          shuffle = False) # test 셋은 shuffle 시행 안함 


In [19]:
# class 활용해서 모델 정의 
class LogisticRegression(nn.Module): 
    def __init__(self, input_size, num_classes):  
        super(LogisticRegression, self).__init__()  # super-class인 nn.Module의 __init__불러오기 
        self.linear = nn.Linear(input_size, num_classes) # 단층 propagation 
  
    def forward(self, x): 
        out = self.linear(x) # out: y_prediction 
        return out 
    
# 모델 초기화     
model = LogisticRegression(input_size, num_classes) 

In [20]:
# Loss 함수 정의 
criterion = nn.CrossEntropyLoss() 

# optimizer 설정 
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate) 

In [25]:
# Training the Model 
for epoch in range(num_epochs): 
    for i, (images, labels) in enumerate(train_loader): 
        images = Variable(images.view(-1, 28 * 28)) 
        labels = Variable(labels) 
  
        # Forward + Backward + Optimize 
        optimizer.zero_grad() # 모든 gradient 0으로 
        outputs = model(images) 
        loss = criterion(outputs, labels)  #loss 계산 
        # 여기까지는 forward pass
        
        loss.backward() #back-propagation
        optimizer.step() #weight 업데이트 
  
        if (i + 1) % 100 == 0: 
            print('Epoch: [% d/% d], Step: [% d/% d], Loss: %.4f'
                  % (epoch + 1, num_epochs, i + 1, 
                     len(train_dataset) // batch_size, loss.data)) 


Epoch: [ 1/ 5], Step: [ 100/ 600], Loss: 2.0283
Epoch: [ 1/ 5], Step: [ 200/ 600], Loss: 1.9679
Epoch: [ 1/ 5], Step: [ 300/ 600], Loss: 1.8286
Epoch: [ 1/ 5], Step: [ 400/ 600], Loss: 1.7713
Epoch: [ 1/ 5], Step: [ 500/ 600], Loss: 1.7940
Epoch: [ 1/ 5], Step: [ 600/ 600], Loss: 1.6702
Epoch: [ 2/ 5], Step: [ 100/ 600], Loss: 1.5771
Epoch: [ 2/ 5], Step: [ 200/ 600], Loss: 1.5821
Epoch: [ 2/ 5], Step: [ 300/ 600], Loss: 1.4949
Epoch: [ 2/ 5], Step: [ 400/ 600], Loss: 1.4766
Epoch: [ 2/ 5], Step: [ 500/ 600], Loss: 1.4371
Epoch: [ 2/ 5], Step: [ 600/ 600], Loss: 1.4347
Epoch: [ 3/ 5], Step: [ 100/ 600], Loss: 1.3263
Epoch: [ 3/ 5], Step: [ 200/ 600], Loss: 1.2704
Epoch: [ 3/ 5], Step: [ 300/ 600], Loss: 1.2623
Epoch: [ 3/ 5], Step: [ 400/ 600], Loss: 1.3267
Epoch: [ 3/ 5], Step: [ 500/ 600], Loss: 1.1119
Epoch: [ 3/ 5], Step: [ 600/ 600], Loss: 1.2522
Epoch: [ 4/ 5], Step: [ 100/ 600], Loss: 1.1629
Epoch: [ 4/ 5], Step: [ 200/ 600], Loss: 1.1822
Epoch: [ 4/ 5], Step: [ 300/ 600], Loss:

In [26]:
# Test the Model 
correct = 0 
total = 0
for images, labels in test_loader: 
    images = Variable(images.view(-1, 28 * 28)) 
    outputs = model(images) 
    _, predicted = torch.max(outputs.data, 1) 
    total += labels.size(0) 
    correct += (predicted == labels).sum() 
  
print('Accuracy of the model on the 10000 test images: % d %%' % ( 
            100 * correct / total)) 


Accuracy of the model on the 10000 test images:  82 %


## Q4-1)
```optim.SGD()```는 모델을 최적화하기 위한 확률적 경사 하강법의 함수다. PyTorch에서는 이와 같이 자동적으로 gradient를 계산 해주지만, 해당 코드를 사용하지 않고 코드를 짠다면 NumPy를 사용하면 된다. <br>
※ 참고로 PyTorch는 NumPy와 매우 유사하지만 GPU를 사용하여 수치 연산을 가속화할 수 있다는 점에서 좀 더 효율적이다. <br>

In [34]:
def gradient(x, y): 
    return 2 * x * (x * w - y)

위와 같이 gradient함수를 NumPy연산으로 정의한 후, 이를 활용해서 gradient를 계산하고 weight를 업데이트하면 된다. 

## Q4-2)
참고: https://wjddyd66.github.io/dl/NeuralNetwork-(3)-Optimazation2/#optimazation-%EA%B3%A0%EB%A0%A4%EC%82%AC%ED%95%AD 

In [35]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [36]:
# For reproducibility
torch.manual_seed(1)

### - SGD + momentum
**momentum**은 Local Minima에 덜빠지기 위해 직전에 계산된 기울기를 고려해서 새로 계산된 기울기와 일정한 비율로 계산하는 것이다.

In [43]:
# 데이터
x_train = torch.FloatTensor([[1], [2], [3]])
y_train = torch.FloatTensor([[1], [2], [3]])

class LinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(1, 1)

    def forward(self, x):
        return self.linear(x)
    
# 모델 초기화
model = LinearRegressionModel()

# optimizer 설정
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):
    
    # H(x) 계산
    prediction = model(x_train)
    
    # cost 계산
    cost = F.mse_loss(prediction, y_train)
    
    # cost로 H(x) 개선
    optimizer.zero_grad() # 미분값이 
    cost.backward()
    optimizer.step()
    
    # 100번마다 로그 출력
    if epoch % 100 == 0:
        params = list(model.parameters())
        W = params[0].item()
        b = params[1].item()
        print('Epoch {:4d}/{} W: {:.3f}, b: {:.3f} Cost: {:.6f}'.format(
            epoch, nb_epochs, W, b, cost.item()
        ))

Epoch    0/1000 W: 0.440, b: -0.357 Cost: 3.021716
Epoch  100/1000 W: 1.005, b: -0.003 Cost: 0.000059
Epoch  200/1000 W: 1.000, b: -0.000 Cost: 0.000000
Epoch  300/1000 W: 1.000, b: -0.000 Cost: 0.000000
Epoch  400/1000 W: 1.000, b: -0.000 Cost: 0.000000
Epoch  500/1000 W: 1.000, b: -0.000 Cost: 0.000000
Epoch  600/1000 W: 1.000, b: -0.000 Cost: 0.000000
Epoch  700/1000 W: 1.000, b: -0.000 Cost: 0.000000
Epoch  800/1000 W: 1.000, b: -0.000 Cost: 0.000000
Epoch  900/1000 W: 1.000, b: -0.000 Cost: 0.000000
Epoch 1000/1000 W: 1.000, b: -0.000 Cost: 0.000000


### - RMSProp
- ***RMSProp**은 Adagrad의 단점을 해결하기 위한 방법이다.
 - Adagrad는 변수들을 업데이트할때 각 변수마다 step size를 다르게 설정해서 이동하는 방식이다. <br>
 *'지금까지 많이 변화하지 않은 변수들은 step size를 크게, 많이 변화해왔으면 이제는 step size를 작게'* <br>
 왜냐면 적게 변화한 경우 그만큼 optimum에 도달하려면 더 많이 이동해야하기 때문이다. 

In [46]:
# 데이터
x_train = torch.FloatTensor([[1], [2], [3]])
y_train = torch.FloatTensor([[1], [2], [3]])

class LinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(1, 1)

    def forward(self, x):
        return self.linear(x)
    
# 모델 초기화
model = LinearRegressionModel()

# optimizer 설정
optimizer = optim.RMSprop(model.parameters(), lr=0.01)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):
    
    # H(x) 계산
    prediction = model(x_train)
    
    # cost 계산
    cost = F.mse_loss(prediction, y_train)
    
    # cost로 H(x) 개선
    optimizer.zero_grad() # 미분값이 
    cost.backward()
    optimizer.step()
    
    # 100번마다 로그 출력
    if epoch % 100 == 0:
        params = list(model.parameters())
        W = params[0].item()
        b = params[1].item()
        print('Epoch {:4d}/{} W: {:.3f}, b: {:.3f} Cost: {:.6f}'.format(
            epoch, nb_epochs, W, b, cost.item()
        ))

Epoch    0/1000 W: 0.774, b: 0.211 Cost: 0.014099
Epoch  100/1000 W: 0.999, b: 0.003 Cost: 0.000001
Epoch  200/1000 W: 1.000, b: -0.000 Cost: 0.000000
Epoch  300/1000 W: 1.000, b: -0.000 Cost: 0.000001
Epoch  400/1000 W: 0.999, b: -0.001 Cost: 0.000016
Epoch  500/1000 W: 0.998, b: -0.002 Cost: 0.000056
Epoch  600/1000 W: 0.997, b: -0.003 Cost: 0.000107
Epoch  700/1000 W: 0.995, b: -0.005 Cost: 0.000262
Epoch  800/1000 W: 0.994, b: -0.006 Cost: 0.000313
Epoch  900/1000 W: 0.995, b: -0.005 Cost: 0.000227
Epoch 1000/1000 W: 0.995, b: -0.005 Cost: 0.000218


### - Adam 
- **Adam**(Adaptive Moment Estimation)은 RMSProp과 Momentum을 합친 것 같은 알고리즘이다. Momentum과 비슷하게 이전에 계산해온 기울기의 지수평균을 저장하며, RMSProp과 유사하게 기울기의 제곱값의 지수평균을 저장한다. <br>
- 다만 초기에는 weight 업데이트 속도가 느리다는 단점이 있다. 

In [47]:
# 데이터
x_train = torch.FloatTensor([[1], [2], [3]])
y_train = torch.FloatTensor([[1], [2], [3]])

class LinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(1, 1)

    def forward(self, x):
        return self.linear(x)
    
# 모델 초기화
model = LinearRegressionModel()

# optimizer 설정
optimizer = optim.Adam(model.parameters(), lr=0.01)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):
    
    # H(x) 계산
    prediction = model(x_train)
    
    # cost 계산
    cost = F.mse_loss(prediction, y_train)
    
    # cost로 H(x) 개선
    optimizer.zero_grad() # 미분값이 
    cost.backward()
    optimizer.step()
    
    # 100번마다 로그 출력
    if epoch % 100 == 0:
        params = list(model.parameters())
        W = params[0].item()
        b = params[1].item()
        print('Epoch {:4d}/{} W: {:.3f}, b: {:.3f} Cost: {:.6f}'.format(
            epoch, nb_epochs, W, b, cost.item()
        ))

Epoch    0/1000 W: -0.362, b: -0.594 Cost: 12.469952
Epoch  100/1000 W: 0.442, b: 0.204 Cost: 1.070746
Epoch  200/1000 W: 0.733, b: 0.462 Cost: 0.053716
Epoch  300/1000 W: 0.787, b: 0.466 Cost: 0.031823
Epoch  400/1000 W: 0.808, b: 0.427 Cost: 0.026483
Epoch  500/1000 W: 0.827, b: 0.384 Cost: 0.021406
Epoch  600/1000 W: 0.847, b: 0.340 Cost: 0.016803
Epoch  700/1000 W: 0.866, b: 0.297 Cost: 0.012817
Epoch  800/1000 W: 0.885, b: 0.255 Cost: 0.009499
Epoch  900/1000 W: 0.902, b: 0.217 Cost: 0.006840
Epoch 1000/1000 W: 0.918, b: 0.181 Cost: 0.004782
